# R workshop: <font color=blue> Generating KEGG Pathway view </font>

#### MLBI@DKU

### __0. Install required R packages__

In [1]:
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
# BiocManager::install("biocLite")
BiocManager::install()

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Installing package(s) 'BiocVersion'



In [2]:
install.packages("reticulate")
BiocManager::install("org.Hs.eg.db")
BiocManager::install("org.Mm.eg.db")
BiocManager::install("biomaRt")
BiocManager::install("gageData")
BiocManager::install("gage")
BiocManager::install("pathview")
install.packages("filesstrings")
install.packages("anndata")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘RcppTOML’, ‘here’, ‘png’


'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Installing package(s) 'org.Hs.eg.db'

also installing the dependencies ‘UCSC.utils’, ‘GenomeInfoDbData’, ‘XVector’, ‘GenomeInfoDb’, ‘plogr’, ‘Biostrings’, ‘BiocGenerics’, ‘Biobase’, ‘IRanges’, ‘RSQLite’, ‘S4Vectors’, ‘KEGGREST’, ‘AnnotationDbi’


'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Installing package(s) 'org.Mm.eg.db'

'getOption("repos")' replaces Bioconductor 

### __1. Load libraries and data__

In [3]:
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(filesstrings))
suppressPackageStartupMessages(library(pathview))
suppressPackageStartupMessages(library(gage))
suppressPackageStartupMessages(library(gageData))
suppressPackageStartupMessages(library(org.Hs.eg.db))
suppressPackageStartupMessages(library(org.Mm.eg.db))
suppressPackageStartupMessages(library(reticulate))
suppressPackageStartupMessages(library(anndata))

In [4]:
reticulate::py_require("gdown")
reticulate::py_require("anndata")

In [5]:
## Install and load "KEGGPathviewGen4SCODA" package
devtools::install_github("combio-dku/KEGGPathviewGen4SCODA")
suppressPackageStartupMessages(library(KEGGPathviewGen4SCODA))

Skipping 5 packages not available: org.Mm.eg.db, org.Hs.eg.db, gageData, gage, pathview



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpFYqKB8/remotes1718424260/combio-dku-KEGGPathviewGen4SCODA-db92b7d/DESCRIPTION’ ... OK
* preparing ‘KEGGPathviewGen4SCODA’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Omitted ‘LazyData’ from DESCRIPTION
* building ‘KEGGPathviewGen4SCODA_0.0.2.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [6]:
## Required to download SCODA-processed sample data
system('pip install mlbi-datasets --upgrade')
reticulate::py_require("mlbi_datasets")
mlbi <- import("mlbi_datasets")

### __2A. Download a preprocessed data__

In [7]:
adata_t <- mlbi$load_scoda_processed_sample_data( 'Colitis-hs' )
adata_t

AnnData object with n_obs × n_vars = 76200 × 18017
    obs: 'celltype', 'nGene', 'nUMI', 'Subject', 'Health', 'Location', 'Sample', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'celltype_org', 'sample', 'condition', 'sample_ext', 'celltype_major', 'celltype_minor', 'celltype_subset'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'variable_genes'
    uns: 'CCI', 'CCI_sample', 'Celltype_marker_DB', 'DEG', 'DEG_grouping_vars', 'DEG_stat', 'DEG_vs_ref', 'DEG_vs_ref_stat', 'GSA_down', 'GSA_up', 'GSA_vs_ref_down', 'GSA_vs_ref_up', 'GSEA', 'GSEA_vs_ref', 'HiCAT_summary', 'Pathways_DB', 'analysis_parameters', 'log', 'lut_sample_to_cond', 'usr_param'
    obsm: 'HiCAT_result', 'X_pca'

### __2AB. Or, Load your scoda-processed data__

In [ ]:
### Load data & extract cell-gene matrix as a data.frame (rownames: cell barcode, colnames: gene symbol)
file_h5ad <- 'colon_hs_SCP259_88K_scoda.h5ad'

adata_t <- read_h5ad(file_h5ad)
adata_t

### __3. Get KEGG pathway IDs and names__

In [8]:
df_pathways_map <- get_pathways_map( adata_t, min_overlap = 0.85 )

  |======================================================================| 100%
Converting Pathways DB .. done.        


Warning message in load(url("https://pathview.uncc.edu/data/khier.rda")):
“URL 'https://pathview.uncc.edu/data/khier.rda': Timeout of 60 seconds was reached”


Error in load(url("https://pathview.uncc.edu/data/khier.rda")) : 
  cannot open the connection to 'https://pathview.uncc.edu/data/khier.rda'


### __4. Generate KEGG pathviews of a single target__

In [9]:
## Check celltype list
target_list <- names(adata_t$uns[['DEG']])
target_list

[1] "B cell"                     "Dendritic cell"            
 [3] "Endothelial cell"           "Enteric glial cell"        
 [5] "Fibroblast"                 "ILC"                       
 [7] "Intestinal Epithelial cell" "Macrophage"                
 [9] "Mast cell"                  "NK cell"                   
[11] "Plasma cell"                "T cell CD4+"               
[13] "T cell CD8+"

In [ ]:
## Check pathways list
df_pathways_map[,'pw_name']

In [11]:
## Select items to generate pathview
items <- c('NOD-like receptor signaling pathway', 'Toll-like receptor signaling pathway',
           'NF-kappa B signaling pathway', 'TNF signaling pathway', 'IL-17 signaling pathwaya')

In [12]:
target_cell <- 'Macrophage'
dir_saved <- save_kegg_pathviews( adata_t, target_cell, df_pathways_map, pathways = items,
                                  deg.p.val.cutoff = 1e-2, gsa.p.val.cutoff = 1e-2,
                                  deg.key = 'DEG_vs_ref', gsa.key = 'GSA_vs_ref_up' )

Macrophage: Healthy_vs_others(2399), Inflamed_vs_Healthy(1807), Non-inflamed_vs_Healthy(2305)
  Healthy_vs_others: No pathways matches with the pathways in the GSA_vs_ref_up. 
  Inflamed_vs_Healthy: 5 intersection with 98 in DB -> 4 
  |======================================================================| 100%
  Non-inflamed_vs_Healthy: 5 intersection with 121 in DB -> 4 
  |======================================================================| 100%
Macrophage: 3/3 - 4/4 - TNF signaling pathway                                      
